### Frame Classifier Final

This notebook creates the final frame classification using the best model we have observed:

- ResNet with LSTM (x3)

In [0]:
import numpy as np
import random
import cv2
from os import listdir
from os.path import isfile, join
import tensorflow as tf

class FramesDataGenerator(tf.keras.utils.Sequence):
    """
    Custom Keras generator for frames
    """
    def __init__(self, dir, batch_size=32, frames_to_use=12, is_test=False, shuffle=True, height=320, width=480):
        self.frames_to_use = frames_to_use
        self.height = height
        self.width = width
        self.batch_size = batch_size
        self.is_test = is_test
        self.dir = dir
        self.shuffle = shuffle
        self.classes = self.find_classes()
        self.video_names, self.video_map, self.video_to_class, self.num_samples, self.min_frames = self.find_samples()
        self.on_epoch_end()
        if self.is_test:
            print(f"Found {self.num_samples} frames belonging to {len(self.video_names)} videos (test-mode).")
        else:
            print(f"Found {self.num_samples} frames belonging to {len(self.video_names)} videos belonging to {len(self.classes)} classes.")
        print(f"Min frames determined to be {self.min_frames}")

    def find_classes(self):
        if self.is_test:
            return []
        else:
            category_folders = [f for f in listdir(self.dir) if not isfile(join(self.dir, f))]
            return sorted(list(set(category_folders)))

    def find_samples(self):
        num_samples = 0
        min_frames = -1
        video_map = {}
        vid_to_cat = {}
        if self.is_test:
            category_folders = [self.dir]
        else:
            category_folders = [f for f in listdir(self.dir) if not isfile(join(self.dir, f))]
        for category_folder in category_folders:
            if self.is_test:
                cat_path = category_folder
            else:
                cat_path = join(self.dir, category_folder)
            frames = [f for f in listdir(cat_path) if isfile(join(cat_path, f))]
            for frame in frames:
                # frame = frame_101_7.mp4_8.jpg
                frame_arr = frame.split(".mp4_")
                vid_name = frame_arr[0]
                if vid_name not in video_map:
                    video_map[vid_name] = []
                    vid_to_cat[vid_name] = category_folder
                video_map[vid_name].append(frame)
            
            for k in video_map.keys():
                # make sure the frames for each video are in sorted order
                video_map[vid_name] = sorted(video_map[vid_name])
                if min_frames == -1 or len(video_map[vid_name]) < min_frames:
                    min_frames = len(video_map[vid_name])

        return list(video_map.keys()), video_map, vid_to_cat, len(vid_to_cat), min_frames

    def __len__(self):
        """
        Denotes the number of batches per epoch
        """
        return int(np.floor(self.num_samples / self.batch_size))

    def __getitem__(self, index):
        """
        Generate one batch of data
        """
        video_names = self.video_names[index*self.batch_size:(index+1)*self.batch_size]
        num_frames = self.min_frames if self.frames_to_use == -1 else self.frames_to_use
        X = np.zeros((len(video_names), num_frames, self.height, self.width, 3), dtype=np.uint8)
        y = []
        i = 0
        for vid in video_names:
            j = 0
            for frame in self.video_map[vid]:
                if self.is_test:
                    frame_path = join(self.dir, frame)
                else:
                    frame_path = join(join(self.dir, self.video_to_class[vid]), frame)

                img = cv2.imread(frame_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                X[i, j, :, :, :] = img
                j += 1
                if j >= num_frames:
                    break

            if self.is_test:
                y.append(0)
            else:
                y.append(int(self.video_to_class[vid]) - 1)
            i += 1
        y = np.array(y)
        return X, tf.keras.utils.to_categorical(y, num_classes=len(self.classes))

    def on_epoch_end(self):
        if self.shuffle == True:
            np.random.shuffle(self.video_names)


In [0]:
import zipfile
import cv2
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
import os
from os.path import isfile, join
import tensorflow as tf
import tempfile
import shutil


class FramesClassifier:
    """
    Classifies sentiment based on frames extracted from video clips
    """

    def __init__(self, frames_folder, model_location=None, is_test=None, is_zip=True, frames_to_use=12, batch_size=16):
        """
        @param frames_folder  The folder where the list of frames are stored. If 
                              `is_zip` is set to True, this should be a single zip 
                              file containing the frames. Paths can either by a local 
                              folder or a GDrive mounted path.
        @param model_location The pre-trained model to perform predictions
        @param is_test        If set to True, we assume that `frames_folder` contains a flat
                              list of videos. If False, we assume that `frames_folder` first 
                              contains subdirectories corresponding to category labels. 
        @param is_zip         If set to True, the `frames_folder` will be unzipped prior to accessing
        @param frames_to_use  The number of frames to use per video
        @param batch_size     The batch size used to feed into the model evaluation
        """
        self.is_zip = is_zip
        self.frames_folder = frames_folder
        self.is_test = is_test
        self.model_location = model_location
        self.frames_to_use = frames_to_use
        self.batch_size = batch_size
        print(f"FramesClassifier created with is_zip = {is_zip}, frames_folder = {frames_folder} , is_test = {is_test} , model_location = {model_location}")

    def predict(self, layer=None):
        folder = self.unzip_folder()
        generator = FramesDataGenerator(folder, is_test=self.is_test, frames_to_use=self.frames_to_use, batch_size=self.batch_size)

        if "https://" in self.model_location or "http://" in self.model_location:
            downloaded_model_path = tf.keras.utils.get_file("frame-classifier", self.model_location)
            model = tf.keras.models.load_model(downloaded_model_path)
        else:
            model = tf.keras.models.load_model(self.model_location)
        if layer is not None:
            print(f"Customizing model by returning layer {layer}")
            model = tf.keras.models.Model(model.input, model.get_layer(layer).output)
        return model.predict(generator)

    def summary(self):
        model = tf.keras.models.load_model(self.model_location)
        model.summary()

    def evaluate(self):
        if self.is_test:
            print("Evaluation cannot be done in test-mode")
            return

        folder = self.unzip_folder()
        generator = FramesDataGenerator(folder, is_test=self.is_test, frames_to_use=self.frames_to_use, batch_size=self.batch_size)
        model = tf.keras.models.load_model(self.model_location)
        return model.evaluate(generator)

    def unzip_folder(self):
        if self.is_zip:
              # Unzips files to a temp directory
              tmp_output_folder = "frames_tmp"
              if os.path.exists(tmp_output_folder) and os.path.isdir(tmp_output_folder):
                  print("Removing existing dir...")
                  shutil.rmtree(tmp_output_folder)

              print(f"Unzipping files to temp dir {tmp_output_folder}...")
              Path(f"{tmp_output_folder}").mkdir(parents=True, exist_ok=True)
              with zipfile.ZipFile(self.frames_folder, 'r') as zip_ref:
                  zip_ref.extractall(tmp_output_folder)
              print("Finished unzipping files")
        else:
            tmp_output_folder = self.frames_folder
            print("Skipping unzipping files as input is a folder")
        return tmp_output_folder




In [36]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
HOME_DIR = "drive/My Drive/"
frames_classifier = FramesClassifier(
    frames_folder = HOME_DIR + "cs231n-project/datasets/emotiw/train-tiny-local.zip", 
    is_test = False,
    model_location = HOME_DIR + "cs231n-project/models/frame-classifier-resnet-lstm-x3.h5"
)
frames_classifier.summary()

FramesClassifier created with is_zip = True, frames_folder = drive/My Drive/cs231n-project/datasets/emotiw/train-tiny-local.zip , is_test = False , model_location = drive/My Drive/cs231n-project/models/frame-classifier-resnet-lstm-x3.h5
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 12, 320, 480 0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 12, 10, 15, 2 23587712    input_4[0][0]                    
__________________________________________________________________________________________________
conv_lst_m2d_3 (ConvLSTM2D)     (None, 12, 10, 15, 4 3006880     time_distributed_1[0][0]         
_____________________________________________________

In [51]:
HOME_DIR = "drive/My Drive/"
frames_classifier = FramesClassifier(
    frames_folder = HOME_DIR + "cs231n-project/datasets/emotiw/train-tiny-local.zip", 
    is_test = False,
    model_location = HOME_DIR + "cs231n-project/models/frame-classifier-resnet-lstm-x3.h5"
)
frames_classifier.evaluate()

FramesClassifier created with is_zip = True, frames_folder = drive/My Drive/cs231n-project/datasets/emotiw/train-tiny-local.zip , is_test = False , model_location = drive/My Drive/cs231n-project/models/frame-classifier-resnet-lstm-x3.h5
Removing existing dir...
Unzipping files to temp dir frames_tmp...
Finished unzipping files
Found 50 frames belonging to 50 videos belonging to 3 classes.
Min frames determined to be 13
3/3 [==============================] - 8s 3s/step - loss: 0.8295 - accuracy: 0.7083


[0.8294854760169983, 0.7083333134651184]

In [52]:
HOME_DIR = "drive/My Drive/"
frames_classifier = FramesClassifier(
    frames_folder = HOME_DIR + "cs231n-project/datasets/emotiw/test-tiny-local.zip", 
    is_test = True,
    model_location = "https://storage.googleapis.com/cs231n-emotiw/frame-classifier-resnet-lstm-x3.h5"
)
frames_classifier.predict()

FramesClassifier created with is_zip = True, frames_folder = drive/My Drive/cs231n-project/datasets/emotiw/test-tiny-local.zip , is_test = True , model_location = https://storage.googleapis.com/cs231n-emotiw/frame-classifier-resnet-lstm-x3.h5
Removing existing dir...
Unzipping files to temp dir frames_tmp...
Finished unzipping files
Found 50 frames belonging to 50 videos (test-mode).
Min frames determined to be 13
202964992/202962720 [==============================] - 2s 0us/step


array([[0.34820428, 0.54439634, 0.10739935],
       [0.2909088 , 0.10773491, 0.6013563 ],
       [0.54589367, 0.21219504, 0.24191126],
       [0.37305444, 0.45109972, 0.17584579],
       [0.22389376, 0.11809638, 0.6580098 ],
       [0.44518152, 0.47764522, 0.07717329],
       [0.39797345, 0.45777172, 0.14425491],
       [0.22224568, 0.10284888, 0.6749054 ],
       [0.61116457, 0.17026111, 0.21857433],
       [0.5346621 , 0.41566825, 0.04966953],
       [0.53102344, 0.19083473, 0.27814186],
       [0.28158262, 0.64193964, 0.07647772],
       [0.3075645 , 0.271059  , 0.4213765 ],
       [0.23010482, 0.28360716, 0.48628804],
       [0.2536567 , 0.08958629, 0.65675706],
       [0.3567246 , 0.51081026, 0.13246517],
       [0.51665413, 0.25464636, 0.22869956],
       [0.22562155, 0.13477427, 0.6396042 ],
       [0.41928637, 0.1851027 , 0.3956109 ],
       [0.2986142 , 0.1822068 , 0.51917905],
       [0.43796587, 0.5088172 , 0.0532169 ],
       [0.36305773, 0.5407344 , 0.09620781],
       [0.

In [45]:
HOME_DIR = "drive/My Drive/"
frames_classifier = FramesClassifier(
    frames_folder = HOME_DIR + "cs231n-project/datasets/emotiw/test-tiny-local.zip", 
    is_test = True,
    model_location = HOME_DIR + "cs231n-project/models/frame-classifier-resnet-lstm-x3.h5"
)
frames_classifier.predict("concatenate_1")

FramesClassifier created with is_zip = True, frames_folder = drive/My Drive/cs231n-project/datasets/emotiw/test-tiny-local.zip , is_test = True , model_location = drive/My Drive/cs231n-project/models/frame-classifier-resnet-lstm-x3.h5
Removing existing dir...
Unzipping files to temp dir frames_tmp...
Finished unzipping files
Found 50 frames belonging to 50 videos (test-mode).
Min frames determined to be 13
Customizing model by returning layer concatenate_1


array([[-1.8647227e-03, -3.4038827e-02,  4.3547651e-01, ...,
         1.7179893e-03,  8.8167608e-01, -3.2296131e-05],
       [-0.0000000e+00, -8.2707131e-01,  8.4936090e-02, ...,
         1.3595320e-03,  9.7490811e-01, -0.0000000e+00],
       [ 3.3392468e-03, -3.6282107e-01,  5.5189729e-01, ...,
         5.7444915e-02,  7.9138809e-01,  0.0000000e+00],
       ...,
       [-0.0000000e+00, -8.0778056e-01,  3.8071179e-01, ...,
         5.0528154e-02,  9.6614629e-01,  0.0000000e+00],
       [-7.2590366e-04, -8.2647181e-01,  2.7033949e-01, ...,
         5.3205504e-03,  9.3420750e-01,  0.0000000e+00],
       [-6.7448891e-03, -6.7763430e-01,  4.9857229e-01, ...,
         5.2003324e-02,  9.1682047e-01,  0.0000000e+00]], dtype=float32)

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
